### Level 1 questions: Solve step by step

### First look for types of tools required for each question

#### 1. load dataset

In [11]:
import os
from datasets import load_dataset

def load_gaia_dataset(levels=["2023_level1"], loader_path="./GAIA.py", split="validation"):
    """
    Load the GAIA dataset for the specified levels.
    
    Args:
        levels: List of dataset levels to process
        loader_path: Path to the GAIA loader script
        split: Dataset split to use
    
    Returns:
        Dictionary containing loaded datasets by level
    """
    loaded_datasets = {}
    
    print(f"Loading GAIA dataset for levels: {levels}")
    
    for level in levels:
        print(f"\nLoading level: {level}")
        
        try:
            dataset = load_dataset(loader_path, name=level, split=split)
            loaded_datasets[level] = dataset
            print(f"Successfully loaded {len(dataset)} examples from {level}")
        except Exception as e:
            print(f"Error loading dataset {level}: {e}")
    
    return loaded_datasets

# Example usage:
levels = ["2023_level1"]  # Change as needed
loader_path = "./GAIA.py"
split = "validation"

datasets = load_gaia_dataset(levels, loader_path, split)

Loading GAIA dataset for levels: ['2023_level1']

Loading level: 2023_level1
Successfully loaded 53 examples from 2023_level1


#### 2. load examples (for val)

In [12]:
from tqdm import tqdm

def get_examples_from_dataset(datasets):
    """
    Extract all examples from loaded datasets.
    
    Args:
        datasets: Dictionary containing loaded datasets by level
    
    Returns:
        List of all examples
    """
    all_examples = []
    
    for level, dataset in datasets.items():
        print(f"Extracting examples from level: {level}")
        
        for i, example in tqdm(enumerate(dataset), total=len(dataset)):
            all_examples.append({
                "level": level,
                "index": i,
                "example": example
            })
    
    print(f"Total examples extracted: {len(all_examples)}")
    return all_examples

# Example usage - requires output from previous step:
examples = get_examples_from_dataset(datasets)
examples

Extracting examples from level: 2023_level1


100%|██████████| 53/53 [00:00<00:00, 9788.56it/s]

Total examples extracted: 53


[{'level': '2023_level1',
  'index': 0,
  'example': {'task_id': 'e1fc63a2-da7a-432f-be78-7c4a95598703',
   'Question': 'If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.',
   'Level': '1',
   'Final answer': '17',
   'file_name': '',
   'file_path': '',
   'Annotator Metadata': {'Steps': '1. Googled Eliud Kipchoge marathon pace to find 4min 37sec/mile\n2. Converted into fractions of hours.\n3. Found moon periapsis in miles (225,623 miles).\n4. Multiplied the two to find the number of hours and rounded to the nearest 100 hours.',
    'Number of steps': '4',
    'How long did this take?': '20 Minutes',
    'Tools': '1. A web browser.\n2. A search 

#### 3. extract ques and answers one by one from examples

In [13]:
def extract_question_and_answer(example):
    """
    Extract question and expected answer from an example.
    
    Args:
        example: Dictionary containing the example data
    
    Returns:
        Dictionary with extracted question and expected answer
    """
    question = example["example"]["Question"]
    expected_answer = example["example"].get("Final answer", "")
    
    return {
        "level": example["level"],
        "index": example["index"],
        "task_id": example["example"].get("task_id", ""),
        "question": question,
        "expected_answer": expected_answer
    }

# Example usage - requires output from previous step:
extracted_data = []
for example in examples:
    extracted_data.append(extract_question_and_answer(example))

# Print the first extracted question and answer
if extracted_data:
    print(f"First example - Question: {extracted_data[0]['question']}")
    print(f"First example - Expected Answer: {extracted_data[0]['expected_answer']}")

First example - Question: If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.
First example - Expected Answer: 17


#### 4. method to read associated file content (some examples have a file attached)

In [5]:
def read_file(file_path):
    """
    Read content from a file.
    
    Args:
        file_path: Path to the file to read
    
    Returns:
        File content as string
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        # Try with a different encoding if UTF-8 fails
        with open(file_path, 'r', encoding='latin-1') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

def get_file_content(example):
    """
    Get file content for an example if available.
    
    Args:
        example: Dictionary containing example data with extracted information
    
    Returns:
        Dictionary with original data plus file content
    """
    # Get file path information from the original example
    original_example = examples[example["index"]]["example"]
    file_path = original_example.get("file_path", "")
    file_name = original_example.get("file_name", "")
    
    file_content = ""
    # Try to read from file_path
    if file_path and os.path.exists(file_path):
        try:
            file_content = read_file(file_path)
            print(f"Read file content from {file_path}: {len(file_content)} characters")
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
    # If that fails, try with file_name
    elif file_name and os.path.exists(file_name):
        try:
            file_content = read_file(file_name)
            print(f"Read file content from {file_name}: {len(file_content)} characters")
        except Exception as e:
            print(f"Error reading file {file_name}: {e}")
    
    # Add file content to the result
    result = example.copy()
    result["file_content"] = file_content
    result["has_file_content"] = bool(file_content)
    
    return result

# Example usage - requires output from previous steps:
examples_with_files = []
for example in extracted_data:
    examples_with_files.append(get_file_content(example))

# Check how many examples have file content
file_count = sum(1 for example in examples_with_files if example["has_file_content"])
print(f"Examples with file content: {file_count} out of {len(examples_with_files)}")

Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/cffe0e32-c9a6-4c52-9877-78ceb4aaa9fb.docx: 17525 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/5cfb274c-0207-4aa7-9575-6ac0bd95d9b2.xlsx: 5115 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/cca530fc-4052-43b2-b130-b30968d8aa44.png: 63079 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/9318445f-fe6a-4e1b-acbf-c68228c9906a.png: 133565 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/389793a7-ca17-4e82-81cb-2b3a2391b4b9.txt: 97 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/a3fbeb63-0e8c-4a11-bff6-0e3b484c3e9c.pptx: 388988 characters
Read file content from /mnt/data4/home/rrao/projects/GaiaBenchmarkHF/2023/validation/65afbc8a-89ca-4ad5-8d62-355bb401f61d.xlsx: 12370 c

#### 5. example of web search

In [2]:
from utils.search_capability import *
from utils.search_capability import search_and_parse

results = search_and_parse("eliud kipchoge nationality", max_results = 3)
print(f"Found {len(results['search_results'])} results.")
print(f"Content length: {len(results['parsed_content'])} characters")

Searching for: eliud kipchoge nationality
Searching with DDGS...
DDGS error: https://html.duckduckgo.com/html 202 Ratelimit
Searching with googlesearch-python...
Found 3 results with googlesearch-python.
Extracting content from search results...
Found 3 results.
Content length: 94163 characters


##### 6. Summarize web search

In [17]:
# Iterate through all the example questions with proper formatting
for example in extracted_data:
    print(f"Task ID: {example['task_id']}")
    print(f"Question: {example['question']}")
    print(f"Expected Answer: {example['expected_answer']}")


Task ID: e1fc63a2-da7a-432f-be78-7c4a95598703
Question: If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.
Expected Answer: 17
Task ID: 8e867cd7-cff9-4e6c-867a-ff5ddc2550be
Question: How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.
Expected Answer: 3
Task ID: ec09fa32-d03f-4bf8-84b0-1f16922c3ae4
Question: Here's a fun riddle that I think you'll enjoy.

You have been selected to play the final round of the hit new game show "Pick That Ping-Pong". In this round, you will be competing for a large cash prize. Your job will be to pick one of several differ

In [ ]:
import ollama
import json
import time

# Assuming extracted_data is already defined and contains a list of dictionaries
# Each dictionary has 'task_id', 'question', and 'expected_answer' keys

# Create a new dictionary to store questions and their proposed tools
results = {}

# Define the prompt template
prompt_template = """You are an AI assistant tasked with determining the tools required to answer a given question from the GAIA benchmark.

Available Tools:
- WEB SEARCH: For retrieving up-to-date or external information.
- CALCULATOR: For performing mathematical computations.
- CODE EXECUTOR: For writing and executing code snippets.

Instructions:
1. Analyze the question carefully.
2. Identify which of the above tools are necessary to answer the question.
3. Provide a Python list containing the names of the required tools. If no tools are needed, return an empty list.

Example:
Question: "What is the capital of France?"
Answer: ["WEB SEARCH"]

Question: "Calculate the sum of 123 and 456."
Answer: ["CALCULATOR"]

Question: "Write a Python function to sort a list of numbers."
Answer: ["CODE EXECUTOR"]

Question: "Explain the process of photosynthesis."
Answer: []

Now, analyze the following question:

Question: "{question}"
Answer: """

# Iterate over each example in extracted_data
for example in extracted_data:
    task_id = example['task_id']
    question = example['question']
    expected_answer = example['expected_answer']
    
    print("\n" + "="*80)
    print(f"Processing Task ID: {task_id}")
    print(f"Question: {question}")
    
    # Format the prompt with the current question
    formatted_prompt = prompt_template.format(question=question)
    
    # Send the prompt to the model
    try:
        response = ollama.chat(
            model='llama3.1:8b',
            messages=[
                {'role': 'user', 'content': formatted_prompt}
            ]
        )
        
        # Extract the model's response
        model_response = response['message']['content'].strip()
        
        # Print the model's response
        print("\nLLM Output:")
        print(model_response)
        print('--'*20)
        # Store the results
        results[task_id] = {
            'question': question,
            'expected_answer': expected_answer,
            'model_response': model_response,
            # You might want to parse the model_response to extract the actual tools list
            # This would depend on how consistently the model formats its responses
        }
        
        # Add a small delay to avoid overwhelming the API
        time.sleep(1)
        
    except Exception as e:
        print(f"\nError processing task {task_id}: {str(e)}")
        # Store the error in the results
        results[task_id] = {
            'question': question,
            'expected_answer': expected_answer,
            'model_response': f"ERROR: {str(e)}",
        }

# Optional: Save the results to a JSON file
with open("gaia_tool_results.json", "w") as file:
    json.dump(results, file, indent=4)

print("\n" + "="*80)
print(f"Processed {len(results)} questions and saved results to 'gaia_tool_results.json'")

Processing Task ID: e1fc63a2-da7a-432f-be78-7c4a95598703
Question: If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many thousand hours would it take him to run the distance between the Earth and the Moon its closest approach? Please use the minimum perigee value on the Wikipedia page for the Moon when carrying out your calculation. Round your result to the nearest 1000 hours and do not use any comma separators if necessary.
Processing Task ID: 8e867cd7-cff9-4e6c-867a-ff5ddc2550be
Question: How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.
Processing Task ID: ec09fa32-d03f-4bf8-84b0-1f16922c3ae4
Question: Here's a fun riddle that I think you'll enjoy.

You have been selected to play the final round of the hit new game show "Pick That Ping-Pong". In this round, you will be competing for a large cash prize. Your job will be to pick one of several different nu

KeyboardInterrupt: 